In [1]:
#Vikas Bansal
#24 July 2020
#This script is used to read the metric file from Cell Ranger ATAC and plot heatmaps (nUMI, nCells etc) for each sample

In [3]:
set.seed(7860)
.libPaths( c( "~/Rlib/", .libPaths()) )
library(readxl)
library(stringi)
library(ComplexHeatmap)
library(dplyr)

In [4]:

#Path to metric files from cell ranger output 
Gene_results_file_names <- list.files(path="/home/vikas/QC_FOUNDIN_scATAC/Metrics_Summary/",recursive = T, full=T)



In [5]:
#Read metric files from cell ranger output
test_count <- read.csv(Gene_results_file_names[4], stringsAsFactors=F, header=T)
rownames(test_count)[1] <- strsplit(unlist(lapply(strsplit(Gene_results_file_names[1],"/"),tail,3)),"_count")[[1]][1]
test_count$SampleName <- rownames(test_count)[1]
ALL_WITH_COUNT <- test_count
for (i in 2:length(Gene_results_file_names)){
  #cat("loop", i, "\n")
  
  
  test_count <- read.csv(Gene_results_file_names[i], stringsAsFactors=F, header=T)
  
  rownames(test_count)[1] <- strsplit(unlist(lapply(strsplit(Gene_results_file_names[i],"/"),tail,3)),"_count")[[1]][1]
  test_count$SampleName <- rownames(test_count)[1]
    ALL_WITH_COUNT <- merge(ALL_WITH_COUNT,test_count,all=T)
  
}


In [6]:
rownames(ALL_WITH_COUNT) <- ALL_WITH_COUNT$SampleName

In [7]:
#convert columns into numeric and save to new dataframe
ALL_WITH_COUNT_tmp <- ALL_WITH_COUNT[,-c(match(c("cellranger.atac_version","SampleName"),colnames(ALL_WITH_COUNT)))]
ALL_WITH_COUNTv2 <- sapply( ALL_WITH_COUNT_tmp, as.numeric )
rownames(ALL_WITH_COUNTv2) <- rownames(ALL_WITH_COUNT_tmp)

In [8]:
#Read metaInfo for samples to keep track of batches
Petertable <- read_excel("Samples_names_codes_final_v.Cornelis.xlsx", sheet = 1)
Petertablev2 <- (as.data.frame(Petertable))
Petertablev3 <- Petertablev2[grep("CDI",Petertablev2[,"Barcode_DZNE"]),]


Petertablev3$Barcode_last4 <- (stri_sub(Petertablev3$Barcode_DZNE,-4,-1))

Petertablev3$SampleID <- (paste0("SCAT_PPMI",Petertablev3$PPMI_ID, "_", Petertablev3$Barcode_last4,"_da65"))


In [9]:
#Put annotation for multiple control samples (SCRN_PPMI3966_2813_da65). Also change the batch column
Petertablev4 <- Petertablev3
Petertablev4[c(nrow(Petertablev3)+1,nrow(Petertablev3)+2),] <- Petertablev3[grep("SCAT_PPMI3966_2813_da65",Petertablev3$SampleID),]
Petertablev4[c(nrow(Petertablev3)+1,nrow(Petertablev3)+2),"SampleID"] <- rownames(ALL_WITH_COUNTv2)[which(is.na(match(rownames(ALL_WITH_COUNTv2),Petertablev3$SampleID)))]
Petertablev4[grep("SCAT_PPMI3966_2813_da65_E6",Petertablev4$SampleID),"BATCH"] <- 5
Petertablev4[grep("SCAT_PPMI3966_2813_da65_E8",Petertablev4$SampleID),"BATCH"] <- 5


In [10]:
tail(Petertablev4)

,PPMI_ID,RECRUITMENT_CAT,IMAGING_CAT,ENROLL_CAT,DESCRP_CAT,genetic_sex,pheno,Barcode_DZNE,Alternate MRN,IID,mutation,ethnicity,BATCH,Barcode_last4,SampleID
94,52932,GENUN,GENUN,GENUN,GBA+,2,1,CDI00019091,ST-00050717,PPMISI52932,GBA_N409S,European,4,9091,SCAT_PPMI52932_9091_da65
95,55124,GENPD,GENPD,GENPD,GBA+,2,2,CDI00021128,ST-00050891,PPMISI55124,GBA_N409S,European,5,1128,SCAT_PPMI55124_1128_da65
96,53988,GENPD,GENPD,GENPD,GBA+,1,2,CDI00018554,ST-00050660,PPMISI53988,GBA_N409S,European,5,8554,SCAT_PPMI53988_8554_da65
97,51844,GENPD,GENPD,GENPD,GBA+,2,2,CDI00019766,ST-00050471,PPMISI51844,GBA_N409S,European,4,9766,SCAT_PPMI51844_9766_da65
98,3966,HC,HC,HC,na,1,1,CDI00012813,ST-00019368,PPMISI3966,na,European,5,2813,SCAT_PPMI3966_2813_da65_E6
99,3966,HC,HC,HC,na,1,1,CDI00012813,ST-00019368,PPMISI3966,na,European,5,2813,SCAT_PPMI3966_2813_da65_E8


In [11]:
#Creating metadata in same order to plot it over heatmap
metaDataHeatmap <- Petertablev4[match(rownames(ALL_WITH_COUNTv2),Petertablev4$SampleID),]




In [12]:
#Scaling the dataframe column wise, meaning for each variable (not sample wise)
ALL_WITH_COUNTv2_scaled = t(scale(ALL_WITH_COUNTv2))

In [13]:
#Creating upper annotation for heatmap
ha = HeatmapAnnotation(Batch = as.factor(metaDataHeatmap$BATCH), 
    pheno = as.factor(metaDataHeatmap$pheno),
    RECRUITMENT_CAT = metaDataHeatmap$RECRUITMENT_CAT,
    genetic_sex = as.factor(metaDataHeatmap$genetic_sex),
    mutation = metaDataHeatmap$mutation, col=list(Batch=c("1"="Red","2"="Blue","3"="Grey","4"="Purple","5"="Black"),
                                                 pheno=c("-9"='#0B5390FF',"1"='#708605FF',"2"='#089B17FF'),
                                                 RECRUITMENT_CAT=c("GENPD"='#F24C5CFF',"GENUN"='#55D5C0FF',"HC"='#E81BF8FF',"PD"='#A11FCEFF',"PRODROMA"='#51ED99FF'),
                                                 genetic_sex=c("1"='#4814C9FF',"2"='#EA6774FF'),
                                                 mutation=c("GBA_N409S"='#F166E7FF',"LRRK2_Arg1441Gly"='#08276FFF',"LRRK2_G2019S"='#8E7719FF',"na"='grey',"SNCA_A53T"='#515152FF'))

)


In [14]:
pdf("HeatmapNumberStats.pdf", width=30, height=10)

Heatmap(ALL_WITH_COUNTv2_scaled, name = "Z-score", km = 1, top_annotation = ha, 
     show_row_names = TRUE, 
    show_column_names = TRUE, show_row_dend = FALSE, cluster_rows = FALSE, cluster_row_slices = FALSE)
dev.off()

png 
  2

In [15]:
sessionInfo()

R version 3.6.1 (2019-07-05)
Platform: x86_64-conda_cos6-linux-gnu (64-bit)
Running under: Ubuntu 16.04.5 LTS

Matrix products: default
BLAS/LAPACK: /opt/anaconda3/lib/R/lib/libRblas.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] grid      stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
[1] dplyr_0.8.0.1        ComplexHeatmap_2.2.0 stringi_1.4.3       
[4] readxl_1.3.1        

loaded via a namespace (and not attached):
 [1] Rcpp_1.0.1          cellranger_1.1.0    compiler_3.6.1     
 [4] pillar_1.3.1        RColorBrewer_1.1-2  base64enc_0.1-3    
 [7] tools_3.6.1         digest_0.6.18       u

In [16]:
ALL_WITH_COUNTv3 <- as.data.frame(t(ALL_WITH_COUNTv2))
ALL_WITH_COUNTv3$Mean <- apply(as.data.frame(t(ALL_WITH_COUNTv2)),1,mean)
ALL_WITH_COUNTv3$Median <- apply(as.data.frame(t(ALL_WITH_COUNTv2)),1,median)
ALL_WITH_COUNTv3$Max <- apply(as.data.frame(t(ALL_WITH_COUNTv2)),1,max)
ALL_WITH_COUNTv3$Min <- apply(as.data.frame(t(ALL_WITH_COUNTv2)),1,min)
ALL_WITH_COUNTv3$SD <- apply(as.data.frame(t(ALL_WITH_COUNTv2)),1,sd)



In [17]:
write.table(t(ALL_WITH_COUNTv3),file="All_metrics_Summary.csv", sep=",", quote = F)